1. Create a Polygon Class:
    - where initializer takes in:
        - number of edges/vertices
        - circumradius
    - that can provide these properties:
        - edges
        - vertices
        - interior angle
        - edge length
        - apothem
        - area
        - perimeter
    - that has these functionalities:
        - a proper __repr__ function
        - implements equality (==) based on # vertices and circumradius (__eq__)
        - implements > based on number of vertices only (__gt__)

In [259]:
class polygon:
    import math
    def __init__(self, num_edges:int, circumrad:float):
        '''
        Accepts 2 parameters - Number of edges(n) and Circumradius(R)
        Derives below properties:
        self.int_angle -> (n−2) * 180 * n 
        self.edge_len  -> s = 2 * R * sin(π/n)
        self.apothem   -> a = R * cos(π * n)
        self.area      -> 12 * n * s * a
        self.perimeter -> n * s
        '''
        if isinstance(num_edges, int) and num_edges >=3:
            self.n                = num_edges
        else:
            raise ValueError('Number of edges must be integer')
        
        if isinstance(circumrad, (int, float)) and circumrad > 0:
            self.R                = circumrad
        else:
            raise ValueError('Circumradius must be a +ve float or integer')                         
               
       
    def __repr__(self):
        '''
        repr function for polygon class. Will give info on
        No: of edges, Circumradius
        '''
        s1 = 'Regular Polygon class having '
        s2 = f'{self.n} sides, circumradius = {self.R}'
        return s1 + s2 
    
    def __eq__(self, oth_poly:'polygon object'):
        '''
        Checks whether a given polygon object is equal or not based on no:of edges and circumradius
        '''
        return self.n == oth_poly.n and self.R == oth_poly.R   
    
    def __gt__(self, oth_poly:'polygon object'):
        '''
        Checks whether a given polygon object is greater than or not based on no:of edges
        '''
        return self.n > oth_poly.n   
    
    @property
    def int_angle(self)-> float:
        return (self.n - 2) * 180/(self.n)
    
    @property
    def edge_len(self)-> float:
        return 2 * self.R * math.sin(math.pi/self.n)
    
    @property
    def apothem(self)-> float:
        return self.R * math.cos(math.pi/self.n)
    
    @property
    def area(self)-> float:
        return 0.5 * self.n * self.edge_len * self.apothem
    
    @property
    def perimeter(self)-> float:
        return self.n * self.edge_len

In [267]:
p1 = polygon(6, 10)
print(round(p1.int_angle,4))
print(round(p1.edge_len,4))
print(round(p1.apothem,4))
print(round(p1.area,4))
print(round(p1.perimeter,4))

120.0
10.0
8.6603
259.8076
60.0


In [277]:
rect = polygon(15, 10)
rect1 = polygon(15,10)

In [278]:
rect == rect1

True

In [279]:
rect2 = polygon(4, 10)
rect > rect2

True

In [276]:
print(rect.n, rect2.n)

15 4


In [282]:
rect3 = polygon(30, 10)
rect < rect3

True

In [281]:
trian = polygon(3, -10.5)

ValueError: Circumradius must be a +ve float or integer

In [264]:
rect

Regular Polygon class having 15 sides, circumradius = 10

In [265]:
repr(rect)

'Regular Polygon class having 15 sides, circumradius = 10'

In [283]:
class poly_seq:
    '''
    This is a ploygon sequence class that can accept:
        - number of vertices for largest polygon in the sequence
        - common circumradius for all polygons
    '''
    import math
    from functools import lru_cache
    
    def __init__(self, edges:int, radius:'int or float')->None:
        '''
        Initializer method
        self.edges -> Input
        self.R     -> Input
        self.eff_ratio -> Will get populated while calling max_efficient()
        '''       
        if isinstance(edges, int) and edges > 0:
            self.edges = edges
        else:
            raise ValueError('Number of vertices for largest polygon in the sequence must be integer > 0')        
        
        if isinstance(radius, (int, float)) and radius > 0:
            self.R = radius
        else:
            raise ValueError('Common Circumradius must be float or integer > 0')
        
        self.eff_ratio = {}
        
    def __repr__(self):
        '''
        repr function for poly_seq class. Will give info on
        No: of edges(n) of largest polyon in the sequence and common Circumradius(R)
        '''
        s1 = 'poly_seq class instance : Largest Polygon class in the sequence has '
        s2 = f'{self.edges} sides with a common circumradius = {self.R}'
        return s1 + s2   
    
    def __len__(self):
        '''
        Returns length of polygon sequence
        '''
        return self.edges
            
    def __getitem__(self, idx:int)->tuple:
        '''
        getitem method that will help us to call the polygon sequences created by index as below
        s1 = poly_seq(6, 10)
        s1[-1], s1[0], s[5] etc.
        
        getitem based on the index it received, calls method '_calc_ratio' by supplying index+1 (to avoid 0 sides) & 
        circumradius used while creating poly_seq class.
        '''
        # This is to handle -ve indexes like -1 s1[-1] should return last element in sequence
        if idx < 0:                
            idx = self.edges + idx
        
        # Here s < 0 is important because this will handle large negative 's' like s = -9999    
        if idx < 0 or idx >= self.edges: 
            raise ValueError(f'Idx Unavailable. For no: of edges = {self.edges}, available indexes are 0 to {int(self.edges-1)}')           
 
        # for indexes 0 & 1 i.e. sides = 1 and sides =2 return area/perimeter ratio as 0 as they are not polygons
        if idx < 2: 
            ratio = 0
        else:
            ratio = poly_seq._calc_ratio(idx+1, self.R) 
            
        msg   = f'Area-Perimeter ration for polygon of side {idx+1} is {ratio}'
        return msg, ratio         
    
                    
    @staticmethod #Static methods are methods that are bound to a class rather than its object.
    @lru_cache(2**10)  ##powers of 2            
    def _calc_ratio(num_edges:int, c_radius:'int or float')->float:
        '''
        This is a method attached to class rather than object. This means we can call this method for parameters that are 
        not used while creating object.
        eg: s1 = poly_seq(6, 10). Here sides = 6, radius =10.
        However we can call s1._calc_ratio(10, 20) ie for sides of 10 and radius = 20 without issues as we are not using self here.
        We are using lru_cache to store the values already claculated so that repetitive calculations can be avoided.
        This method is directly called in __getitem__() and indirectly called from max_efficient() methods.        
        '''
        poly = polygon(num_edges, c_radius)
        return poly.area/poly.perimeter         
            
    @property
    def max_efficient(self)->str:
        '''
        This method returns the Polygon with the highest area: perimeter ratio.
        Calls _getitem__ for each edge starting from 0 till self.edges -1 .
        __getitem__ fetches area-perimeter ratio then by calling _calc_ratio.
        If n = 0, __getitem__ gives area-per ratio for side = 1
           n = 5, __getitem__ gives area-per ratio for side = 6        
        '''
    
        for n in range(self.edges):
            self.eff_ratio[n+1] = self.__getitem__(n)[1]
            
        max_eff = max(self.eff_ratio, key=self.eff_ratio.get) 
        # Reference for 'max' usage with 'key': https://stackoverflow.com/questions/18296755/python-max-function-using-key-and-lambda-expression

        s1 = f'Polygon with max efficiency for circumradius of {self.R} is one with {max_eff} sides & '
        s2 = f'Area-perimeter ratio for the same is {round(self.eff_ratio.get(max_eff), 4)}'
        return s1 + s2           
        

In [284]:
s1 = poly_seq(6, 10)

In [290]:
repr(s1)

'poly_seq class instance : Largest Polygon class in the sequence has 6 sides with a common circumradius = 10'

In [286]:
len(s1)

6

In [287]:
for i in s1:
    print(i)

('Area-Perimeter ration for polygon of side 1 is 0', 0)
('Area-Perimeter ration for polygon of side 2 is 0', 0)
('Area-Perimeter ration for polygon of side 3 is 2.5000000000000004', 2.5000000000000004)
('Area-Perimeter ration for polygon of side 4 is 3.535533905932738', 3.535533905932738)
('Area-Perimeter ration for polygon of side 5 is 4.045084971874737', 4.045084971874737)
('Area-Perimeter ration for polygon of side 6 is 4.330127018922194', 4.330127018922194)


ValueError: Idx Unavailable. For no: of edges = 6, available indexes are 0 to 5

In [289]:
s1[0]

('Area-Perimeter ration for polygon of side 1 is 0', 0)

In [233]:
s1[2]

('Area-Perimeter ration for polygon of side 3 is 2.5000000000000004',
 2.5000000000000004)

In [234]:
s1[3]

('Area-Perimeter ration for polygon of side 4 is 3.535533905932738',
 3.535533905932738)

In [235]:
s1[5]

('Area-Perimeter ration for polygon of side 6 is 4.330127018922194',
 4.330127018922194)

In [236]:
s1[6]

ValueError: Idx Unavailable. For no: of edges = 6, available indexes are 0 to 5

In [237]:
s1[-1]

('Area-Perimeter ration for polygon of side 6 is 4.330127018922194',
 4.330127018922194)

In [238]:
s1.max_efficient

'Polygon with max efficiency for circumradius of 10 is one with 6 sides & Area-perimeter ratio for the same is 4.3301'

In [239]:
s1._calc_ratio(10, 20)

9.510565162951535

In [240]:
s1.max_efficient

'Polygon with max efficiency for circumradius of 10 is one with 6 sides & Area-perimeter ratio for the same is 4.3301'

In [241]:
poly_seq.max_efficient

In [242]:
s2 = poly_seq(edges=25, radius=10)

In [243]:
s2.max_efficient

'Polygon with max efficiency for circumradius of 10 is one with 25 sides & Area-perimeter ratio for the same is 4.9606'